In [1]:
import os
os.listdir('../workspace/Tomato/Test')

['Bacterial_spot',
 'Early_blight',
 'Healthy',
 'Late_blight',
 'Leaf_Mold',
 'Mosaic_virus',
 'Septoria_leaf_spot',
 'Spider_mites',
 'Target_Spot',
 'Yellow_Leaf_Curl_Virus']

In [2]:
from tensorflow import keras
from keras.layers import Layer
import keras.backend as K
import tensorflow as tf


from keras.datasets import cifar10
from keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, BatchNormalization, Activation, MaxPooling2D, GlobalMaxPooling2D,\
    Reshape,  multiply, add, Permute

import math
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2

2024-03-13 21:03:34.403905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 21:03:34.404024: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 21:03:34.404131: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 21:03:34.446545: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from tensorflow import keras
from keras.layers import Layer
import keras.backend as K
import tensorflow as tf


from keras.datasets import cifar10
from keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, BatchNormalization, Activation, MaxPooling2D, GlobalMaxPooling2D,\
    Reshape,  multiply, add, Permute

import math
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2

In [4]:
import tensorflow as tf
import imageio
import numpy as np


import os
import imageio
import numpy as np

def read_images(dir):
    supported_extensions = [".jpg", ".jpeg", ".png"]
    image_list = []
    count = 0
    # Walk through the directory and read images
    for root, _, files in os.walk(dir):
        for file in files:
            file_extension = os.path.splitext(file)[-1].lower()

            # Check if the file is a supported image format
            if file_extension in supported_extensions:
                image_path = os.path.join(root, file)
                try:
                    image = imageio.imread(image_path)
                    image = np.asarray(image)

                    # If the image has a fourth channel, remove it
                    if image.shape[-1] == 4:
                        image = image[:, :, :3]

                    image_list.append(image)
                except Exception as e:
                    print(f"Error reading image {image_path}: {e}")

            count += 1
            if count % 100 == 0:
                print(str(count) + " images read")

    return image_list


In [5]:
import os

# Train data directories
base_train_dir = 'Tomato/Test/'

bacterial_spot = read_images(os.path.join(base_train_dir, 'Bacterial_spot'))
early_blight = read_images(os.path.join(base_train_dir, 'Early_blight'))
healthy = read_images(os.path.join(base_train_dir, 'Healthy'))
late_blight = read_images(os.path.join(base_train_dir, 'Late_blight'))
leaf_mold = read_images(os.path.join(base_train_dir, 'Leaf_Mold'))
mosaic_virus = read_images(os.path.join(base_train_dir, 'Mosaic_virus'))
septoria_leaf_spot = read_images(os.path.join(base_train_dir, 'Septoria_leaf_spot'))
spider_mites = read_images(os.path.join(base_train_dir, 'Spider_mites'))
target_spot = read_images(os.path.join(base_train_dir, 'Target_Spot'))
yellow_leaf_curl_virus = read_images(os.path.join(base_train_dir, 'Yellow_Leaf_Curl_Virus'))


/tmp/ipykernel_116561/1919092470.py:23: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


100 images read
200 images read
300 images read
400 images read
100 images read
200 images read
100 images read
200 images read
300 images read
100 images read
200 images read
300 images read
100 images read
100 images read
200 images read
300 images read
100 images read
200 images read
300 images read
100 images read
200 images read
100 images read
200 images read
300 images read
400 images read
500 images read
600 images read
700 images read
800 images read
900 images read
1000 images read


In [6]:
import numpy as np

# Labeling for new train classes
labels_bacterial_spot = np.zeros(len(bacterial_spot))
labels_early_blight = np.full(len(early_blight), 1)
labels_healthy = np.full(len(healthy), 2)
labels_late_blight = np.full(len(late_blight), 3)
labels_leaf_mold = np.full(len(leaf_mold), 4)
labels_mosaic_virus = np.full(len(mosaic_virus), 5)
labels_septoria_leaf_spot = np.full(len(septoria_leaf_spot), 6)
labels_spider_mites = np.full(len(spider_mites), 7)
labels_target_spot = np.full(len(target_spot), 8)
labels_yellow_leaf_curl_virus = np.full(len(yellow_leaf_curl_virus), 9)

# Combine train labels
labels = np.concatenate([
    labels_bacterial_spot,
    labels_early_blight,
    labels_healthy,
    labels_late_blight,
    labels_leaf_mold,
    labels_mosaic_virus,
    labels_septoria_leaf_spot,
    labels_spider_mites,
    labels_target_spot,
    labels_yellow_leaf_curl_virus
])


In [7]:
(len(bacterial_spot) + len(early_blight) + len(healthy) + 
                len(late_blight) + len(leaf_mold) + len(mosaic_virus) + 
                len(septoria_leaf_spot) + len(spider_mites) + 
                len(target_spot) + len(yellow_leaf_curl_virus))


3637

In [8]:
# Create an array to store the frequencies
class_freq = []

# Append the lengths of the lists to the array
class_freq.append(len(bacterial_spot))
class_freq.append(len(early_blight))
class_freq.append(len(healthy))
class_freq.append(len(late_blight))
class_freq.append(len(leaf_mold))
class_freq.append(len(mosaic_virus))
class_freq.append(len(septoria_leaf_spot))
class_freq.append(len(spider_mites))
class_freq.append(len(target_spot))
class_freq.append(len(yellow_leaf_curl_virus))

# Calculate the sum of all frequencies
total_freq = sum(class_freq)

# Calculate the relative frequencies
relative_freq = [freq / total_freq for freq in class_freq]


In [9]:
images = (bacterial_spot + early_blight + healthy + late_blight + 
          leaf_mold + mosaic_virus + septoria_leaf_spot + spider_mites + 
          target_spot + yellow_leaf_curl_virus)


In [10]:
len(images)

3637

In [11]:
tf.__version__

'2.14.0'

In [12]:
# !pip install tensorflow==2.14

In [13]:
model = keras.models.load_model('../workspace/Model/epoch122_model_part2.keras')

2024-03-13 21:03:48.679927: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [14]:
import imgaug as ia
import imgaug.augmenters as iaa
import random
import copy

# Find accuracy of model
def find_accuracy(test,pred):
    correct = 0
    total = len(test)

    for i in range(len(test)):
        if test[i] == pred[i]:
            correct += 1

    return correct/total


# Map ANN outputs to classes
def get_labels(y_pred_ann):
    labels = []

    for pred in y_pred_ann:
        max_index = 0

        for i in range(len(pred)):
            if pred[i] > pred[max_index]:
                max_index = i

        labels.append(max_index)

    return labels


def resize_images(images_list, width=128, height=128):
    ia.seed(1)
    # Define the resize augmentation
    resize_augmenter = iaa.Resize({"height": height, "width": width})

    resized_images = []

    for image in images_list:
        # Ensure the image is in RGB format (imgaug uses RGB by default)
        if image.shape[-1] == 1:  # Grayscale image with single channel
            image = np.repeat(image, 3, axis=-1)

        # Apply the resize augmentation
        augmented_image = resize_augmenter.augment_image(image)

        # Append the augmented image to the result list
        resized_images.append(augmented_image)

    del images_list[:]
    return resized_images

def normalize_images(image_list):
  for i in range(len(image_list)):
    image = image_list[i].astype(np.float32) / 255.0
    image_list[i] = image

In [15]:

images = resize_images(images, 224, 224)
normalize_images(images)
images = np.array(images)

pred = model.predict(images)

2024-03-13 21:04:26.233523: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-13 21:04:26.246569: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2024-03-13 21:04:26.589090: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


114/114 [==============================] - 9s 49ms/step


In [16]:
n_classes = 10

y_pred = get_labels(pred)
total = len(y_pred)

correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == labels[i]:
        correct += 1

accuracy = correct / total

print(correct)
print(total)

# Initialize variables to store weighted sums
weighted_sum_sensitivity = 0
weighted_sum_specificity = 0
weighted_sum_precision = 0
weighted_sum_accuracy = 0
weighted_sum_f1 = 0

class_metrics = []

for i in range(n_classes):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for j in range(len(y_pred)):
        if labels[j] == i and y_pred[j] == i:
            TP += 1
        elif labels[j] != i and y_pred[j] == i:
            FP += 1
        elif labels[j] == i and y_pred[j] != i:
            FN += 1
        elif labels[j] != i and y_pred[j] != i:
            TN += 1


    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else -1
    specificity = TN / (TN + FP) if (TN + FP) > 0 else -1
    precision = TP / (TP + FP) if (TP + FP) > 0 else -1
    class_accuracy = (TP + TN) / (TP + FN + TN + FP)
    recall = sensitivity
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else -1

    class_metrics.append({
        'class': int(i),
        'TP': TP,
        'FP': FP,
        'TN': TN,
        'FN': FN,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'Precision': precision,
        'Accuracy': class_accuracy,
        'F1 Score': f1,
        'Relative Frequency': relative_freq[i]
    })

    # Calculate weighted sums
    weighted_sum_sensitivity += sensitivity * relative_freq[i]
    weighted_sum_specificity += specificity * relative_freq[i]
    weighted_sum_precision += precision * relative_freq[i]
    weighted_sum_accuracy += class_accuracy * relative_freq[i]
    weighted_sum_f1 += f1 * relative_freq[i]



print(accuracy)


3616
3637
0.9942260104481716


In [17]:
import pandas as pd

def create_metrics_dataframe(class_metrics):
    # Create a dictionary to store the metrics for each class
    metrics_dict = {
        'Class': [metrics['class'] for metrics in class_metrics],
        'Specificity': [metrics['Specificity'] for metrics in class_metrics],
        'Sensitivity': [metrics['Sensitivity'] for metrics in class_metrics],
        'Precision': [metrics['Precision'] for metrics in class_metrics],
        'Accuracy': [metrics['Accuracy'] for metrics in class_metrics],
        'F1 Score': [metrics['F1 Score'] for metrics in class_metrics]
    }

    # Create a Pandas DataFrame from the metrics dictionary
    metrics_df = pd.DataFrame(metrics_dict)

    return metrics_df

# Usage:
# Assuming you have class_metrics list defined as in your previous code
metrics_dataframe = create_metrics_dataframe(class_metrics)

# Calculate weighted average metrics
weighted_avg_metrics = {
    'Class': 'Weighted Avg',
    'Specificity': weighted_sum_specificity,  # Replace with your actual values
    'Sensitivity': weighted_sum_sensitivity,  # Replace with your actual values
    'Precision': weighted_sum_precision,      # Replace with your actual values
    'Accuracy': weighted_sum_accuracy,        # Replace with your actual values
    'F1 Score': weighted_sum_f1               # Replace with your actual values
}

if isinstance(weighted_avg_metrics, dict):
    weighted_avg_metrics_df = pd.DataFrame([weighted_avg_metrics])
else:
    weighted_avg_metrics_df = weighted_avg_metrics

# Concatenate the original dataframe with the weighted_avg_metrics dataframe
metrics_dataframe = pd.concat([metrics_dataframe, weighted_avg_metrics_df], ignore_index=True)
metrics_dataframe = metrics_dataframe.round(3)
print(metrics_dataframe)

# Save the dataframe to a csv file
metrics_dataframe.to_csv('classwise_metrics.csv', index=False)


           Class  Specificity  Sensitivity  Precision  Accuracy  F1 Score
0              0        1.000        0.998      0.998     0.999     0.998
1              1        0.998        0.990      0.966     0.998     0.978
2              2        0.999        1.000      0.994     0.999     0.997
3              3        1.000        0.976      0.997     0.997     0.987
4              4        1.000        1.000      1.000     1.000     1.000
5              5        1.000        1.000      1.000     1.000     1.000
6              6        1.000        0.997      1.000     1.000     0.999
7              7        0.999        0.985      0.991     0.998     0.988
8              8        0.999        0.993      0.982     0.998     0.988
9              9        0.999        0.999      0.998     0.999     0.999
10  Weighted Avg        0.999        0.994      0.994     0.999     0.994


In [18]:
for i in range(len(y_pred)):
  print(y_pred[i], labels[i])

0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
9 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
0 0.